## Step 1: Importing Necessary Libraries

In [ ]:
%pip install -U pandas numpy matplotlib seaborn scikit-learn gzip

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

## Step 2: Loading the Dataset
We are loading the dataset from the provided CSV file and displaying the first 10 rows to get an initial look at the data.

In [ ]:
# Read from local files
df = pd.read_csv('Crime_Data_from_2020_to_Present.csv')

# Display the first 10 rows of the dataset
df.head(10)

## Step 3: Inspecting Columns and Data Types
We are checking the structure of the dataset to understand the columns and their data types.

In [ ]:
# Overview of data types and column names
df.info()

# Checking the first few rows to see column names and data
df.head()

## Step 4: Class Distribution (Crm Cd Desc)
We are examining the class distribution of the crime type (Crm Cd Desc) to see how balanced or imbalanced the data is. A bar plot is created to visualize the top 10 crime types.

In [ ]:
# Class distribution for Crm Cd Desc
class_distribution = df['Crm Cd Desc'].value_counts()

# Display distribution
print(class_distribution)

# Plot the top 10 crime types
plt.figure(figsize=(10,6))
sns.countplot(y='Crm Cd Desc', data=df, order=class_distribution.index[:10])
plt.title('Top 10 Crime Type Distribution')
plt.show()

## Step 5: Checking for Missing Values
We are identifying missing values in the dataset to know which columns require further cleaning.

In [ ]:
missing_values = df.isnull().sum()
print(missing_values)

## Step 6: Descriptive Statistics
We are calculating basic descriptive statistics (mean, median, standard deviation) for the numeric columns to better understand the dataset.

In [ ]:
# Descriptive statistics for numeric columns
df.describe()

## Step 7: Data Visualization - Histogram for Victim Age
We are visualizing the distribution of Vict Age to understand how victim ages are spread and whether there are any outliers.

In [ ]:
# Histogram of Victim Age
plt.figure(figsize=(8,6))
sns.histplot(df['Vict Age'].dropna(), bins=20)
plt.title('Distribution of Victim Age')
plt.show()

## Data Cleaning
Step 1: Handling Missing Values
We are deciding how to handle missing values. Here, we fill the missing values in Vict Age with the median, and we drop rows where the target column (Crm Cd Desc) has missing values.

In [ ]:
# Fill missing values in Vict Age with median
df['Vict Age'].fillna(df['Vict Age'].median(), inplace=True)

# Drop rows with missing Crm Cd Desc (target column)
df.dropna(subset=['Crm Cd Desc'], inplace=True)

# Remove rows where both Latitude and Longitude are 0
df = df[(df['LAT'] != 0) & (df['LON'] != 0)]

## Step 2: Removing Irrelevant Columns
We are identifying and removing columns that do not contribute to the crime type prediction (such as unique IDs).

In [10]:
# Remove irrelevant columns
df.drop(columns=['DR_NO', 'Status Desc'], inplace=True)

## Step 3: Handling Duplicates
We are checking for and removing any duplicate rows in the dataset.

In [11]:
# Drop duplicates
df.drop_duplicates(inplace=True)

## Step 4: Handling Outliers
We are identifying and removing outliers in numeric columns like Vict Age. Here, we visualize outliers using a boxplot and remove any rows where Vict Age is above 100.

In [ ]:
# Visualize outliers using boxplot
plt.figure(figsize=(8,6))
sns.boxplot(df['Vict Age'])
plt.title('Boxplot for Victim Age')
plt.show()

# Remove outliers (Example: removing ages greater than 100)
df = df[df['Vict Age'] <= 100]

## Data Transformation
Step 1: Convert Time and Date to Usable Formats
Convert Date Occ and TIME OCC to datetime format and extract new features like day of the week.

In [ ]:
# Convert DATE OCC to datetime
df['DATE OCC'] = pd.to_datetime(df['DATE OCC'])

# Convert Time Occ to usable format
df['HOUR OCC'] = pd.to_datetime(df['TIME OCC'], format='%H%M', errors='coerce').dt.hour

# Extract useful features like Day of Week, Month, Year
df['Day of Week'] = df['DATE OCC'].dt.day_name()
df['Month'] = df['DATE OCC'].dt.month
df['Year'] = df['DATE OCC'].dt.year

# Create a new feature Crime Period (Day vs. Night)
df['Crime Period'] = df['HOUR OCC'].apply(lambda x: 'Day' if 6 <= x <= 18 else 'Night')



## Step 2: One-Hot Encoding for Categorical Variables
Apply one-hot encoding to columns like Vict Sex, Premis Cd Desc.

In [14]:
# One-hot encode Vict Sex and Premis Cd Desc
df = pd.get_dummies(df, columns=['Vict Sex', 'Premis Desc'], drop_first=True)

## Step 3: Normalization of Victim Age
Normalize the Vict Age column using StandardScaler.

In [15]:
# Normalize Vict Age
scaler = StandardScaler()
df['Vict Age'] = scaler.fit_transform(df[['Vict Age']])

## Step 4: Feature Engineering
Incorporating Holidays into Time Analysis

In [ ]:

%pip install holidays
import holidays

# Add a column indicating whether the crime occurred on a holiday
us_holidays = holidays.US()
df['Holiday'] = df['DATE OCC'].apply(lambda x: 1 if x in us_holidays else 0)

# Visualization of crimes on holidays vs non-holidays
plt.figure(figsize=(10, 6))
sns.countplot(x='Holiday', data=df, palette='Set1')
plt.title('Crime on Holidays vs Non-Holidays')
plt.xticks([0, 1], ['Non-Holiday', 'Holiday'])
plt.show()


# Initial Data Visualization and Analysis
## Step 1: Crime Distribution by Day of the Week
Visualize how crimes are distributed across the days of the week.

In [ ]:
# Plot crime distribution by day of the week
plt.figure(figsize=(10, 6))
sns.countplot(x='Day of Week', data=df, order=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
plt.title('Crime Distribution by Day of the Week')
plt.show()

# Plot crime distribution by hour of the day
plt.figure(figsize=(10, 6))
sns.countplot(x='HOUR OCC', data=df, order=sorted(df['HOUR OCC'].unique()))  # Ensure the hours are in sorted order
plt.title('Crime Distribution by Hour of the Day')
plt.xlabel('Hour (24-Hour Format)')
plt.show()

## Step 2: Geographic Distribution of Crimes
Visualize the geographic distribution of crimes using latitude and longitude.

In [ ]:
# Scatter plot of crime locations (Longitude vs. Latitude)
plt.figure(figsize=(10,6))
sns.scatterplot(x='LON', y='LAT', data=df, alpha=0.5)
plt.title('Geographic Distribution of Crimes')
plt.show()

## Step 3: Correlation Matrix
Visualize the correlation between numeric features.

In [ ]:
# Filter the dataset to include only numeric columns
numeric_df = df.select_dtypes(include=['number'])

# Compute correlation matrix for numeric features
plt.figure(figsize=(12,8))
correlation_matrix = numeric_df.corr()

# Plot the heatmap
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix of Numeric Features')
plt.show()

# Evaluation and Conclusion
---

### **Explanation of Preprocessing Steps**

1. **Handling Missing Values**:
   We identified columns that contained missing values and decided to handle them using the most appropriate method for each case:
   - For columns critical to the analysis, such as `Crm Cd Desc` (Crime Code Description) and `Vict Age` (Victim Age), we removed rows that contained missing values. This decision was based on the importance of these columns for crime type prediction; imputing these values could introduce bias or noise into the dataset.
   - Other columns with missing values that were deemed irrelevant to the crime type prediction task were dropped from the dataset.

2. **Removing Irrelevant Data**:
   Certain columns, such as unique identifiers (`DR_NO`) or redundant columns, were removed from the dataset. These columns were not expected to contribute to the prediction of crime types and were removed to streamline the dataset and improve model performance by reducing noise.

3. **Handling Duplicates**:
   We checked for duplicate entries in the dataset. Duplicate rows, if present, were removed to avoid redundancy and ensure the integrity of the data. This step helps prevent overrepresentation of certain data points, which could skew model predictions.

4. **Handling Outliers**:
   Outliers were identified in numeric columns such as `Vict Age` (Victim Age). We analyzed the distribution of these variables and decided to remove outliers to improve the overall quality of the dataset. Removing outliers helps ensure that the machine learning models are not disproportionately influenced by extreme values that do not reflect typical crime occurrences.

5. **Date and Time Transformation**:
   The `Date Occ` (Date of Occurrence) and `Time Occ` (Time of Occurrence) columns were transformed to extract useful features. We converted these columns into a usable datetime format and created additional features such as `Hour`, `Day of the Week`, and `Month`. This allowed us to capture temporal patterns in crime occurrences, such as the times or days when crimes are more likely to happen.

6. **Categorical Encoding**:
   Categorical variables, such as `Vict Sex` (Victim's Gender), were encoded using one-hot encoding. This transformation allowed us to represent categorical data numerically, which is essential for machine learning models that require numerical inputs.

7. **Normalization and Standardization**:
   Numerical columns like `Vict Age` were normalized to ensure that all features are on a similar scale. This step helps improve the performance of machine learning models by preventing features with larger ranges from dominating the training process.

8. **Final Dataset Preparation**:
   After cleaning, transforming, and encoding the data, the final dataset was saved in a CSV format. This dataset is now free from missing values, irrelevant columns, duplicates, and outliers, and it is ready for use in predictive modeling.


---

### **Summary of Key Patterns from Data Exploration**

1. **Class Distribution**: Certain crime types, such as "Battery" and "Theft," are dominant in the dataset. This indicates a class imbalance that may need to be addressed in later modeling steps.
   
2. **Time-based Patterns**: Crimes are more frequent during late evening hours, particularly between 6 PM to 11 PM, with a noticeable decline during early morning hours. This suggests that the time of day is a critical factor in crime occurrence.

3. **Day of the Week**: There is a significant increase in crimes during weekends, especially on Saturdays. This implies that certain days are more crime-prone than others.

4. **Geospatial Distribution**: The geographical analysis reveals that crimes tend to cluster in certain areas, indicating that some zones are more prone to criminal activity. These geographical zones could provide strong predictive power for crime types.

5. **Victim's Age**: The distribution of victim ages shows a concentration around young to middle-aged adults, with some outliers at older ages. This suggests that age might also play a role in predicting certain types of crimes.

---

### **Relevant Features for Predicting Crime Types**

1. **Time-related Features**: The hour of the day, day of the week, and month are crucial predictors. Different types of crimes show distinct patterns during specific times, days, and months.

2. **Geographical Zones**: Geographic location is a strong predictor, as crimes are not uniformly distributed across the city. The zones derived from clustering the location data highlight areas that are more prone to certain crime types.

3. **Victim’s Demographics**: Attributes such as the victim’s age and gender could help predict the type of crime, given that certain demographics are more commonly targeted in particular types of crimes.

4. **Holiday Indicator**: Whether or not the crime occurred on a holiday could be relevant, as crime patterns tend to shift during special occasions and holidays.

## Save the Cleaned Dataset
Save the cleaned dataset for further modeling.

In [20]:
import gzip

# Save the cleaned dataset to a CSV file with gzip compression
df.to_csv('cleaned_crime_data.csv.gz', index=False, compression='gzip')
